<a href="https://colab.research.google.com/github/sahith2004/Deep-learning-practice/blob/main/LSTM_text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np

# Load and preprocess text data
path_to_file = 'shakespeare.txt'  # Replace with the path to your text file
text = open(path_to_file, 'r').read()
vocab = sorted(set(text))
char2idx = {u: i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
text_as_int = np.array([char2idx[c] for c in text])



In [ ]:
# Create input sequences and target sequences
seq_length = 100
examples_per_epoch = len(text) // (seq_length + 1)
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_length + 1, drop_remainder=True)

def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)




In [ ]:
# Create a LSTM-based model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(len(vocab), 256, batch_input_shape=[64, None]),
    tf.keras.layers.LSTM(1024, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(len(vocab))
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

# Define a checkpoint callback to save model weights
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath="text_generation_model.h5", save_best_only=True)

# Train the model
history = model.fit(dataset, epochs=30, callbacks=[checkpoint_callback])

# Load the best model
model.load_weights("text_generation_model.h5")



In [ ]:
# Generate text
def generate_text(model, start_string, num_generate=1000, temperature=1.0):
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    generated_text = []

    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()
        input_eval = tf.expand_dims([predicted_id], 0)
        generated_text.append(idx2char[predicted_id])

    return start_string + ''.join(generated_text)

# Generate text using a starting prompt
generated_text = generate_text(model, start_string="To be or not to be, that is the question:")
print(generated_text)